<a href="https://colab.research.google.com/github/stbalaji/AIForBusiness/blob/main/05%20Generative%20AI/AICH_AIB_M4_GenAI_Enterprise_RAG_Example02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Retrival Augmented Generation

### Large Language Models (LLMs) provide a host of capabilities today including  Generation of New Content.

### Challenges: However, LLMs have a set of constraints when applying to Specific use cases by Companies - particular the models provide Generic Intelligence but companies / businesses need domain or use case specific capabilities.

### RAG help with tapping the power of LLMs in the Context of a "Knowledge base". As an example, using RAG a company can power a Customer Service Use case where the responses to user issues or service requests are in the context of hte Company (e.g. laptop issues by DELL).

### Reference Paper : https://arxiv.org/abs/2005.11401

## In this Example, we will how RAG can be used to respond in the context of a a web knowledge

In [2]:
!pip install langchain sentence-transformers chromadb
# faiss-cpu

In [4]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings  # will not download but remain in cloud

from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub

from langchain.chains import RetrievalQA, LLMChain

## Setup HuggingFace Access Token

- Log in to [HuggingFace.co](https://huggingface.co/)
- Click on your profile icon at the top-right corner, then choose [“Settings.”](https://huggingface.co/settings/)
- In the left sidebar, navigate to [“Access Token”](https://huggingface.co/settings/tokens)
- Generate a new access token, assigning it the “write” role.

In [7]:
#Hugging Face : Need Access Token
import os
from getpass import getpass

HF_TOKEN = getpass("HF Token:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

HF Token:··········


In [42]:
import nest_asyncio

nest_asyncio.apply()

In [43]:
WEBSITE_URL = "https://businessofinsights.com/"

In [44]:
loader = WebBaseLoader(WEBSITE_URL)
loader.requests_per_second = 1
docs = loader.aload()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.09it/s]


In [45]:
docs

[Document(metadata={'source': 'https://businessofinsights.com/', 'title': 'Business of Insights !  | Let us power your Transformation Journey Together !', 'language': 'en'}, page_content='\n\n\n\n\n\n\nBusiness of Insights !  | Let us power your Transformation Journey Together !\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Me\nCourses\nData Science\nAI\nDigital\nContact\nTOS\nPrivacy\n \n\n\n\n\n\nSelect Page\n\n\n  \n \n\n\n\n \n\n\n\n \n\n\n\n\n\n\n\n\n\n\nWe power your Journey \nto Insights\xa0 & Automation !\n\n\nWe are outcome focussed Insights Practioners who are passionate about data, intelligence and Insights.\xa0\nWe help businesses Transform by leveraging power of Data, Insights and Automation. We also help with skills transformation for you or your teams – helping\xa0 you master breath of Applied Data and Insights capabilities.\nSpeed to Applied Insights !\xa0\n\n\n\n\n\n\n\n\n\n\n\n\xa0 \xa0 \xa0Learn.\xa0 \xa0 Apply.\xa0 \xa0 Scale.\xa

In [47]:
# Break Text into Components - Text Splitter. To avoid repetation etc..
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=20) # Chunk Overlap is important. Begining a Chunk context is also there in the next chuck.
chunks = text_splitter.split_documents(docs)

In [48]:
len(chunks)

12

In [49]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

In [50]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [51]:
vectorstore

In [52]:
llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

In [53]:
query = "What is Business of Insights?"
search = vectorstore.similarity_search(query)

# Step 1 of RAG : Retriever

In [54]:
retriever = vectorstore.as_retriever(
    search_type="mmr", #similarity
    search_kwargs={'k': 2}           # How many relevant documents we want to retrive
)

In [55]:
query = "What are courses offered?"
doc_rels = retriever.get_relevant_documents(query)
doc_rels

[Document(metadata={'language': 'en', 'source': 'https://businessofinsights.com/', 'title': 'Business of Insights !  | Let us power your Transformation Journey Together !'}, page_content='Our Services\xa0\n\n\n\n\n\nProfessional\n Services\n\n\n\n\n\nl\n\nAdvisory\nServices\n\n\n\n\nA\n\nProject \nDelivery\n\n\n\ni\n\nBusiness\n Support\xa0\n\n\n\n\n\n\n\n\n\nLearning \nServices\n\n\n\n\n\n\nl\n\nCorporate\nTraining\xa0\n\n\n\n\nA\n\nUpskilling for\nExperienced\n\n\n\ni\n\nNew EntrantTraining'),
 Document(metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'}, page_content='Blogs:\nMedium: @jaintarun7\nGeeksForGeeks: jaintarun\nAnalytics Vidya: tarun4641039\n\n\n\nNewsletter\nSubscribe to my Newsletter\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConnect\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n          © Copyright Tarun R Jain 2022')]

In [21]:
doc_rels[0]

Document(metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'}, page_content="Data Scientist @AI Planet🥑 ||  AI with Tarun- YouTube🤗||  GDE in AI/ML🚀 ||  GSoC'23 caMicroscope\nKnow more\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Me")

In [22]:
doc_rels[1]

Document(metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app/', 'title': 'Tarun Jain'}, page_content='DevFest Kathmandu, Nepal\n\r\n                 Got invited at Kathmandu Nepal to speak on Retrieval Augmented Generation and Fine Tuning for large language models and code demo')

In [23]:
llm = HuggingFaceHub(
    repo_id="huggingfaceh4/zephyr-7b-alpha",
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

# Step 2 of RAG : Augment

In [24]:
query = "Name the projects Tarun has worked on?"

# System Prompt and User Prompt

prompt = f"""
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

# Step 3 of RAG : Generate

In [25]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever)  #Chain_Type = Stuff or refine etc


In [26]:
response = qa.invoke(prompt)

In [27]:
response

{'query': '\n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n Name the projects Tarun has worked on?\n </s>\n <|assistant|>\n',
 'result': "The original question is as follows: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n Name the projects Tarun has worked on?\n </s>\n <|assistant|>\n\nWe have provided an existing answer: Context information is below. \n------------\nTarun Jain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Me\nEvents\nAchievements\nProjects\nWork\nConnect\n\nResume\n\n\nAI with Tarun\n\n\n\n\n\n\n\n\n\n\n\n\nIt's Me...  Tarun Jain\n------------\nGiven the context information and not prior knowledge, answer the question: \n <|system|>\nYou are an AI assistant that follows instruction extremely well.\nPlease be truthful and give direct answers\n</s>\n <|user|>\n Name the 

In [ ]:
print(response['result'])



Original answer: Tarun has worked on various projects, including GDE in ML, Project Ca Microscope, AI with Tarun, and DevRel at AI Planet.

Refined answer: In addition to the projects mentioned earlier, Tarun has also worked on MangaPi, a software that converts manga pages into digital format. He has also contributed to a research paper on hyperspectral image compression using MATLAB (Hardware), published in the International Journal of Electrical and Computer Engineering. To stay updated on his latest tutorials and resources, you can subscribe to his YouTube channel AI with Tarun.


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
 <|system|>
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
 <|user|>
 {query}
 </s>
 <|assistant|>
"""

In [ ]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

TypeError: unsupported operand type(s) for |: 'dict' and 'str'

In [ ]:
response = rag_chain.invoke("Name the projects Tarun has worked on?")

NameError: name 'rag_chain' is not defined

In [ ]:
print(response)